In [1]:
from pandas import read_csv, DataFrame
import pandas
from sklearn.decomposition import PCA, FastICA
from sklearn.cluster import DBSCAN, MiniBatchKMeans
from IPython.display import display
pandas.set_option('max_rows', 200)
pandas.set_option('max_columns', 60)

In [2]:
raw_counties = read_csv('county_facts.csv').dropna(axis=0, how='any')
raw_counties = raw_counties[raw_counties['PST045214'] > 0]
counties = raw_counties.drop(['fips', 'area_name', 'state_abbreviation'], axis=1)
counties = (counties - counties.min()) / (counties.max() - counties.min())
counties.describe()

,PST045214,PST040210,PST120214,POP010210,AGE135214,AGE295214,AGE775214,SEX255214,RHI125214,RHI225214,RHI325214,RHI425214,RHI525214,RHI625214,RHI725214,RHI825214,POP715213,POP645213,POP815213,EDU635213,EDU685213,VET605213,LFE305213,HSG010214,HSG445213,HSG096213,HSG495213,HSD410213,HSD310213,INC910213,INC110213,PVY020213,BZA010213,BZA110213,BZA115213,NES010213,SBO001207,SBO315207,SBO115207,SBO215207,SBO515207,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210
count,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000
mean,0.010023,0.010000,0.194281,0.009999,0.430450,0.536821,0.276147,0.742235,0.848868,0.108453,0.024401,0.033077,0.002530,0.064991,0.092115,0.774766,0.726386,0.079942,0.096863,0.732463,0.232594,0.020400,0.466755,0.012228,0.769259,0.125889,0.140824,0.011380,0.343320,0.276518,0.253828,0.301682,0.009389,0.009645,0.395921,0.007740,0.008389,0.035847,0.008222,0.014143,0.001522,0.021677,0.319367,0.008389,0.005895,0.010459,0.126647,0.007763,0.008308,0.007712,0.003730
std,0.032241,0.031874,0.046625,0.031873,0.088605,0.081277,0.090977,0.085136,0.172742,0.169612,0.082424,0.064481,0.019741,0.051137,0.141005,0.208352,0.089831,0.099187,0.120633,0.128011,0.123992,0.049083,0.137817,0.035863,0.085743,0.094386,0.083552,0.034421,0.075755,0.104197,0.116595,0.124267,0.032281,0.035639,0.032434,0.029799,0.030140,0.103227,0.053118,0.044913,0.033214,0.079829,0.228715,0.033710,0.035520,0.034732,0.067231,0.035601,0.032384,0.024824,0.024823
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.001082,0.001124,0.166852,0.001124,0.379562,0.488095,0.219262,0.726592,0.798087,0.009401,0.003254,0.009434,0.000000,0.040816,0.019874,0.652618,0.677551,0.021201,0.029289,0.651852,0.147472,0.002690,0.371859,0.001542,0.727079,0.062944,0.090029,0.001300,0.296073,0.207370,0.178143,0.214149,0.000873,0.000554,0.382337,0.000779,0.000863,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000638,0.084805,0.000244,0.000125,0.002947,0.000243
50%,0.002533,0.002627,0.186874,0.002627,0.423358,0.535714,0.270492,0.760300,0.922423,0.028202,0.006508,0.016509,0.002070,0.054422,0.038703,0.852356,0.736735,0.044170,0.052301,0.757407,0.202247,0.006100,0.462312,0.003514,0.782516,0.099492,0.116919,0.003018,0.335347,0.263168,0.236406,0.286807,0.002130,0.001669,0.396050,0.001730,0.002058,0.000000,0.000000,0.000000,0.000000,0.000000,0.412811,0.000000,0.000215,0.002077,0.120390,0.001010,0.000799,0.004220,0.000650
75%,0.006690,0.006793,0.213571,0.006793,0.474453,0.576190,0.323770,0.786517,0.964931,0.126910,0.013829,0.030660,0.002070,0.074830,0.094142,0.942408,0.787755,0.098940,0.107741,0.829630,0.285112,0.016852,0.552764,0.008811,0.827292,0.161421,0.165215,0.007941,0.377644,0.325363,0.305016,0.370937,0.005709,0.005189,0.408667,0.004474,0.005389,0.000000,0.000000,0.000000,0.000000,0.000000,0.494662,0.005295,0.001213,0.006434,0.160037,0.003752,0.003645,0.006337,0.001635
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000

In [3]:
pca = PCA(32)
reduced = pca.fit_transform(counties)
#ica = FastICA(40)
#reduced = ica.fit_transform(counties)

In [4]:
filename = 'county_facts_dictionary.csv'
with open(filename) as f:
    lines = f.readlines()
    col_names = {line.split(',', 1)[0]: line.split(',', 1)[1].strip() for line in lines[1:]}

In [5]:
k=16
c = MiniBatchKMeans(n_clusters=k, random_state=42)
clusters = c.fit_transform(reduced)
labels = clusters.argmin(axis=1)
[list(labels).count(i) for i in range(k)]

[343, 287, 183, 110, 289, 25, 32, 320, 124, 628, 97, 132, 159, 88, 227, 98]

In [6]:
labeled_counties = raw_counties.copy().set_index('fips')
labeled_counties['label'] = labels
labeled_counties.columns = [col_names[column] if column in col_names else column for column in labeled_counties.columns]

In [7]:
raw_votes = read_csv('votes.csv')
vote_diffs = DataFrame({'dem_diff': (raw_votes['votes_dem_2016']/raw_votes['population2014'] 
                                   - raw_votes['votes_dem_2012']/raw_votes['population2010'])*100,
                        'rep_diff': (raw_votes['votes_gop_2016']/raw_votes['population2014'] 
                                   - raw_votes['votes_gop_2012']/raw_votes['population2010'])*100,
                        'trump_margin': 100 * (raw_votes['votes_gop_2016'] - raw_votes['votes_dem_2016'])
                                 / raw_votes['population2014'],
                        'obama_margin': 100 * (raw_votes['votes_dem_2012'] - raw_votes['votes_gop_2012'])
                                 / raw_votes['population2010'],
                        'pop': raw_votes['population2014'],
                        'fips': raw_votes['county_fips']
                       }).set_index('fips')
with_votes = labeled_counties.join(vote_diffs, how='inner')

In [27]:
label_nums = set(labels)
by_label = with_votes.groupby('label')
state_pops = with_votes.groupby("state_abbreviation")["pop"].sum()
for i, group in by_label:
    means = group.mean()
    stds = group.std()
    summary = (means - with_votes.mean())/with_votes.std()
    sig_stats = summary[col_names.values()][summary.abs() > .5]
    print "================ Cluster %i =====================" % i
    print "# of Counties: %i" % list(labels).count(i)
    print "Total population: {:,}".format(group['pop'].sum())
    print "Republican diff: %f +- %f" % (means['rep_diff'], stds['rep_diff'])
    print "Democrat diff: %f +- %f" % (means['dem_diff'], stds['dem_diff'])
    print "Trump margin: %f +- %f" % (means['trump_margin'], stds['trump_margin'])
    print "Obama margin: %f +- %f" % (means['obama_margin'], stds['obama_margin'])
    display(sig_stats.sort_values(ascending=False))
    print "By state:"
    print (group.groupby("state_abbreviation")["pop"].sum()/state_pops).dropna().sort_values(ascending=False)


================ Cluster 0 =====================
# of Counties: 343
Total population: 4,168,579
Republican diff: 3.335179 +- 3.010114
Democrat diff: -4.431324 +- 2.528794
Trump margin: 25.233115 +- 11.291225
Obama margin: -17.466612 +- 13.692383


"High school graduate or higher, percent of persons age 25+, 2009-2013"    0.793541
"White alone, not Hispanic or Latino, percent, 2014"                       0.793207
"Persons 65 years and over, percent, 2014"                                 0.774557
"White alone, percent, 2014"                                               0.670002
"Homeownership rate, 2009-2013"                                            0.525537
"Foreign born persons, percent, 2009-2013"                                -0.509098
"Black or African American alone, percent, 2014"                          -0.583354
"Persons per household, 2009-2013"                                        -0.757792
"Persons below poverty level, percent, 2009-2013"                         -0.782477
"Mean travel time to work (minutes), workers age 16+, 2009-2013"          -0.813219
"Women-owned firms, percent, 2007"                                        -1.328747
dtype: float64

By state:
state_abbreviation
SD    0.159420
NE    0.152064
IA    0.151174
ND    0.122318
MT    0.089296
KS    0.074753
MN    0.046750
IN    0.045568
WI    0.044243
WV    0.037162
IL    0.032956
MI    0.022085
MO    0.020677
WY    0.018892
NY    0.018828
OH    0.018196
PA    0.017463
MD    0.017131
ID    0.013820
VA    0.011793
VT    0.011163
UT    0.009868
KY    0.007013
OR    0.006249
OK    0.005496
CO    0.002461
TX    0.001811
WA    0.001765
NV    0.000711
Name: pop, dtype: float64
================ Cluster 1 =====================
# of Counties: 287
Total population: 20,880,923
Republican diff: 1.881401 +- 2.241128
Democrat diff: -1.851742 +- 1.364647
Trump margin: 13.642103 +- 7.111176
Obama margin: -9.908960 +- 7.288320


"Black or African American alone, percent, 2014"                           0.784840
"Women-owned firms, percent, 2007"                                         0.727525
"High school graduate or higher, percent of persons age 25+, 2009-2013"   -0.549932
"White alone, not Hispanic or Latino, percent, 2014"                      -0.575002
"White alone, percent, 2014"                                              -0.759758
dtype: float64

By state:
state_abbreviation
SC    0.383464
LA    0.382983
MS    0.317729
NC    0.215846
AL    0.209956
OK    0.193610
DE    0.183823
GA    0.158846
ID    0.140450
NM    0.125334
FL    0.109253
OH    0.103639
AR    0.102631
MI    0.094399
TN    0.084313
TX    0.076435
IN    0.074312
AZ    0.070604
IL    0.058450
VA    0.047879
MD    0.027885
KY    0.018243
NJ    0.017609
MT    0.009111
MO    0.008271
NV    0.007671
CO    0.006076
CA    0.005887
OR    0.005590
Name: pop, dtype: float64
================ Cluster 2 =====================
# of Counties: 183
Total population: 5,041,737
Republican diff: 4.668944 +- 2.189724
Democrat diff: -3.024084 +- 1.721429
Trump margin: 23.111667 +- 5.570879
Obama margin: -15.418640 +- 6.878515


...

By state:
state_abbreviation
DC    1.000000
GA    0.395363
LA    0.384301
AL    0.354309
OH    0.284441
MD    0.281435
TN    0.260381
MS    0.219972
SC    0.213085
VA    0.212879
NC    0.202491
MI    0.178085
WI    0.166113
MO    0.165019
AR    0.149088
IN    0.141619
PA    0.122020
FL    0.059401
KS    0.055659
TX    0.009357
Name: pop, dtype: float64
================ Cluster 14 =====================
# of Counties: 227
Total population: 4,402,678
Republican diff: 5.003746 +- 2.288404
Democrat diff: -3.660640 +- 1.903760
Trump margin: 23.752056 +- 5.218679
Obama margin: -15.087670 +- 6.698222


"Mean travel time to work (minutes), workers age 16+, 2009-2013"           0.927886
"White alone, not Hispanic or Latino, percent, 2014"                       0.753079
"Homeownership rate, 2009-2013"                                            0.604842
"White alone, percent, 2014"                                               0.580596
"Language other than English spoken at home, pct age 5+, 2009-2013"       -0.513957
"High school graduate or higher, percent of persons age 25+, 2009-2013"   -0.529695
"Foreign born persons, percent, 2009-2013"                                -0.540000
"Median household income, 2009-2013"                                      -0.607116
"Per capita money income in past 12 months (2013 dollars), 2009-2013"     -0.607438
"Retail sales per capita, 2007"                                           -0.651406
"Housing units in multi-unit structures, percent, 2009-2013"              -0.656090
"Bachelor's degree or higher, percent of persons age 25+, 2009-2013"      -0

By state:
state_abbreviation
WV    0.169698
KY    0.128735
TN    0.080453
MO    0.065488
IN    0.052651
AR    0.052584
AL    0.038326
MS    0.033791
VA    0.027545
MI    0.024240
OH    0.019586
GA    0.016503
IL    0.012927
TX    0.012503
KS    0.011358
IA    0.010689
ID    0.010319
NC    0.009695
LA    0.008378
ND    0.008098
WA    0.008020
MT    0.004483
MN    0.004446
PA    0.003828
FL    0.002385
NY    0.001599
UT    0.000504
Name: pop, dtype: float64
================ Cluster 15 =====================
# of Counties: 98
Total population: 1,809,881
Republican diff: 3.808188 +- 2.713054
Democrat diff: -3.505397 +- 2.496262
Trump margin: 21.396288 +- 11.688973
Obama margin: -14.082703 +- 12.097387


"Persons 65 years and over, percent, 2014"                      1.792280
"Homeownership rate, 2009-2013"                                 0.601717
"Housing units in multi-unit structures, percent, 2009-2013"   -0.550666
"Female persons, percent, 2014"                                -0.713788
"Persons per household, 2009-2013"                             -1.113310
"Women-owned firms, percent, 2007"                             -1.249229
"Persons under 5 years, percent, 2014"                         -1.481119
"Persons under 18 years, percent, 2014"                        -1.590131
dtype: float64

By state:
state_abbreviation
MT    0.098965
FL    0.044279
SD    0.027777
NH    0.023856
WI    0.016136
MI    0.011912
ID    0.010120
CO    0.009455
VA    0.009121
MD    0.008646
ND    0.006421
NC    0.005689
KS    0.005414
OR    0.004560
IL    0.003581
NE    0.003474
MN    0.002890
CA    0.002260
MO    0.002093
TX    0.002016
KY    0.001910
NM    0.001705
OK    0.001493
NV    0.001378
WA    0.001140
PA    0.001084
NY    0.000239
Name: pop, dtype: float64


In [29]:
with_votes[with_votes['state_abbreviation'] == 'NC'][['area_name', 'label', 'rep_diff', 'dem_diff', 'trump_margin', 'obama_margin']].sort_values(by='label')

,area_name,label,rep_diff,dem_diff,trump_margin,obama_margin
fips,,,,,,
37001,Alamance County,1,-0.392699,0.047621,5.755751,-6.196071
37047,Columbus County,1,2.752278,-3.100352,9.105754,-3.253124
37045,Cleveland County,1,3.312032,-1.716382,13.904570,-8.876156
37153,Richmond County,1,2.498014,-2.771864,4.174229,1.095650
37143,Perquimans County,1,2.415821,-3.406019,13.753156,-7.931316
37157,Rockingham County,1,2.121987,-1.938258,13.529489,-9.469244
37061,Duplin County,1,0.925707,-1.564770,6.604656,-4.114178
37159,Rowan County,1,2.678751,-2.417654,16.720768,-11.624363
37141,Pender County,1,3.261401,-1.959791,14.632889,-9.411697
